In [408]:
import sys
sys.path.append('../')

import pandas as pd
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary

In [385]:
fashion_train_filename = '/Users/rimiiii/data/fashion_mnist/fashion-mnist_train.csv'
fashion_val_filename = '/Users/rimiiii/data/fashion_mnist/fashion-mnist_test.csv'

text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import torch
import pandas as pd


class FashionDataset(Dataset):
    def __init__(self, train_filename):
        self.df = pd.read_csv(train_filename)
        self.height = 28
        self.width = 28

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df.iloc[idx][0]
        img = np.array(self.df.iloc[idx][1:])
        img = img.reshape(-1, self.height, self.width)
        return torch.FloatTensor(img), label

In [405]:
import torch.nn as nn
import torch
import torch.nn.functional as F


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 128, 3)
        self.conv3 = nn.Conv2d(128, 32, 3)
        self.fc1 = nn.Linear(32*22*22, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [406]:
dataset = FashionDataset(fashion_train_filename)
loader = DataLoader(dataset, batch_size=32, shuffle=True)
val_dataset = FashionDataset(fashion_val_filename)
val_loader = DataLoader(val_dataset, batch_size=10000, shuffle=False)
net = CNN()
#print(summary(net, input_size=(1, 28, 28, 1), verbose=0))

In [407]:
lr = 0.001
n_epochs = 30

optimizer = optim.Adam(net.parameters(), lr=lr)
#criterion = nn.MultiLabelSoftMarginLoss() ## 공부할 것
criterion = nn.NLLLoss()

total_batch = len(loader)
train_losses = []
for epoch in range(n_epochs):
    total_cost = 0
    for imgs, labels in loader:
        optimizer.zero_grad()

        outputs = net(imgs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        total_cost += loss.item()
        train_losses.append(loss.item())

    avg_loss = total_cost / len(loader)
    print(f'Epoch: {epoch+1} Cost: {avg_loss}')

Epoch: 1 Cost: 0.5424833359003067


KeyboardInterrupt: 

In [295]:
net.eval()
val_dataset = FashionDataset(fashion_val_filename)
val_loader = DataLoader(val_dataset, batch_size=10000, shuffle=False)

# total_cost = 0
# for img, label in val_loader:
#     pred = net(img)
#     loss = criterion(pred, label)
#     print(f'pred: {pred.argmax()}, label: {label}')
#     print(loss.item())
#     total_cost += loss.item()
#
# avg_val_loss = total_cost / len(val_loader)
# print(f'Cost = {avg_val_loss}')
